In [ ]:
!python3 -m pip install --upgrade openai

In [1]:
import os
import openai
import constants
openai.api_key = constants.GPT_API_KEY

In [2]:
dataset_folder = './nerv_dataset'

In [11]:
def get_responses(song_name, genre):
    prompt_1 = f'The string on the line below is the YouTube video title of a piano cover of a(n) {genre} song.\
                 Given the string, write a rich musical description of the original song, not the cover.\
                 This description should be roughly 200 words.\
                 \n{song_name}'
    prompt_2 = f'Now, provide a list of aspects describing the music of the original song, using \",\" as the delimiter,\
                 and without spaces between aspects. Below are three examples:\n\
                 low quality,sustained strings melody,soft female vocal,mellow piano melody,sad,soulful,ballad\n\
                 sustained piano synth,lullaby,bright xylophone,mellow melody,bass guitar,dreamy,kids music,slow tempo,nostalgic,cello,no vocals,sleepy\n\
                 rock,passionate male vocal,harmonizing male vocals,wide electric guitar melody,groovy bass,punchy kick,punchy snare,shimmering cymbals,energetic,addictive'

    response_1 = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
                {"role": "user", "content": prompt_1}
            ]
    )
    caption = response_1['choices'][0]['message']['content'].lstrip('\n')

    response_2 = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
                {"role": "user", "content": prompt_2}
            ]
    )
    aspects = response_2['choices'][0]['message']['content'].lstrip('\n')
    
    return (caption, aspects)

In [7]:
def caption_all_genre_folders(overwrite=False):
    genre_list = os.listdir(dataset_folder)
    for genre in genre_list:
        if genre.startswith('.') or not os.path.isdir(f'{dataset_folder}/{genre}'):
            continue
        
        caption_genre_folder(genre, overwrite)


def caption_genre_folder(genre, overwrite=False):
    '''
    Takes in the name of a genre folder containing .mp3 files and generates rich text captions
    for each song in the genre folder in .txt format

            Parameters:
                    genre (str): the genre whose songs to caption
                    overwrite (bool): specify whether captioning should overwrite existing 
                                      .txt files with the same file name
    '''
    genre_path = f'{dataset_folder}/{genre}'

    song_list = os.listdir(genre_path)

    for song_name in song_list:
        if song_name.startswith('.'):
            continue
        caption_filename = song_name + '_caption.txt'
        aspects_filename = song_name + '_aspects.txt'
        if not overwrite and os.path.isfile(f'{genre_path}/{song_name}/{caption_filename}'):
            continue

        song_path = f'{genre_path}/{song_name}'
        if not os.path.exists(song_path):
            os.mkdir(song_path)

        caption, aspects = get_responses(song_name, genre)
        assert(caption != None and len(caption) != 0)
        assert(aspects != None and len(aspects) != 0)

        with open(f'{song_path}/{caption_filename}', 'w') as f:
            f.write(caption)
        with open(f'{song_path}/{aspects_filename}', 'w') as f:
            f.write(aspects)

In [12]:
caption_all_genre_folders()
